# Step I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import re
import os
import glob
import numpy as np
import json
import csv
import cassandra
import re

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
# print(os.getcwd())

In [3]:
# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    if (re.search(".*event_data$", root)):
    # join the file path and roots with the subdirectories using glob
        file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

In [5]:
# get total number of rows 
print(len(full_data_rows_list))
# check to see what the list of event data rows will look like
# print(full_data_rows_list)

8056


In [6]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [7]:
# check the number of rows in csv file event_datafile_new
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Step II. Modeling the Apache Cassandra database

## Work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the file event_datafile_new.csv after the code above is run:<br>

<img src="images/event_datafile_new.png" style="width:2000px;">

#### Creating a Cluster

In [8]:
# This makes a connection to a Cassandra instance in local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [9]:
try:
    session.execute("create Keyspace if not exists sparkify with replication = {'class': 'SimpleStrategy', 'replication_factor': 1}")
except Exception as e:
    print(e)

#### Set Keyspace

In [10]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Query: show the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [11]:
query = "create table if not exists songs_by_sessionId_itemInSession"
query = query + " (artist_name varchar, itemInSession int, song_length float, sessionId int, song_title varchar, primary key (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)              

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into songs_by_sessionId_itemInSession (artist_name, itemInSession, song_length, sessionId, song_title)"
        query = query + " values (%s, %s, %s, %s, %s)"
        
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

In [13]:
query = "select artist_name, song_title, song_length from songs_by_sessionId_itemInSession where sessionId = 338 and itemInSession = 4"

result = []
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    result.append(row)

df = pd.DataFrame(result)
df.columns = ["artist_name", "song_title", "song_length"]
df

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query: show the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [14]:
query = "create table if not exists history_by_userid_sessionid"
query = query + " (artist_name varchar, user varchar, itemInSession int, sessionId int, song_title varchar, userId int, primary key (sessionId, userId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)              


In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into history_by_userid_sessionid (artist_name, user, itemInSession, sessionId, song_title, userId)"
        query = query + " values (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1] + ' ' + line[4], int(line[3]), int(line[8]), line[9], int(line[10])))

In [16]:
query = "select artist_name, song_title, user from history_by_userid_sessionid where sessionId = 182 and userId = 10"
result = []
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    result.append(row)

df = pd.DataFrame(result)
df.columns = ["artist_name", "song_title", "user"]
df
                    

,artist_name,song_title,user
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,Three Drives,Greece 2000,Sylvie Cruz
2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


### Query: show user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

In [17]:
query = "create table if not exists users_by_songs"
query = query + " (user varchar, song_title varchar, primary key (song_title, user))"
try:
    session.execute(query)
except Exception as e:
    print(e)   
                    

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into users_by_songs (user, song_title)"
        query = query + " values (%s, %s)"
        session.execute(query, (line[1] + ' ' + line[4], line[9]))

In [19]:
query = "select user from users_by_songs where song_title = 'All Hands Against His Own'"
result=[]
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    result.append(row)

df = pd.DataFrame(result)
df.columns = ["user"]
df

,user
0,Jacqueline Lynch
1,Sara Johnson
2,Tegan Levine


### Drop the tables before closing out the sessions

In [20]:
try:
    session.execute("drop table if exists users_by_songs")
except Exception as e:
    print(e)

In [21]:
try:
    session.execute("drop table if exists history_by_userid_sessionid")
except Exception as e:
    print(e)

In [22]:
try:
    session.execute("drop table if exists songs_by_sessionId_itemInSession")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()